In [1]:
!pip install riotwatcher

In [50]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import random
import time

In [3]:
 TIERS = ["IRON", "SILVER", "GOLD", "PLATINUM", "DIAMOND"]
RANKS = ["I", "II", "III", "IV"]
REGIONS = ["eun1"]
PAGES = [1]

In [34]:
class Karma:
  def __init__(self, API_KEY):
    self.API_KEY = API_KEY
    self.lol_watcher = LolWatcher(API_KEY)
  
  def player_stats(self, region, summoner):
    player = self.lol_watcher.summoner.by_name(region, summoner)
    ranked_stats = self.lol_watcher.league.by_summoner(region, player['id'])
    columns = [
        "name",
        "summonerLevel",
        "tierRank",
        "wins",
        "losses",
        "veteran",
        "inactive",
        "freshBlood",
        "hotStreak"
    ]

    data = {}
    # player = lol_watcher.summoner.by_name('eun1', 'spikal')
    data['name'] = player['name']
    data['summonerLevel'] = player['summonerLevel']
    if len(ranked_stats) > 0:
      for i in range(0,len(ranked_stats)):
        if 'RANKED_SOLO_5x5' in ranked_stats[i]['queueType']:
          if 'tier' in ranked_stats[i] and 'rank' in ranked_stats[i]:
            t = ranked_stats[i]['tier'] + ' ' + ranked_stats[i]['rank']
            data['tierRank'] = t
          else:
            data['tierRank'] = None

          if 'wins' in ranked_stats[i]:
            t = ranked_stats[i]['wins']
            data['wins'] = t
          else:
            data['wins'] = None

          if 'losses' in ranked_stats[i]:
            t = ranked_stats[i]['losses']
            data['losses'] = t
          else:
            data['losses'] = None

          if 'veteran' in ranked_stats[i]:
            t = ranked_stats[i]['veteran']
            data['veteran'] = t
          else:
            data['veteran'] = None

          if 'inactive' in ranked_stats[i]:
            t = ranked_stats[i]['inactive']
            data['inactive'] = t
          else:
            data['inactive'] = None

          if 'freshBlood' in ranked_stats[i]:
            t = ranked_stats[i]['freshBlood']
            data['freshBlood'] = t
          else:
            data['freshBlood'] = None

          if 'hotStreak' in ranked_stats[i]:
            t = ranked_stats[i]['hotStreak']
            data['hotStreak'] = t
          else:
            data['hotStreak'] = None
    else:
      data['tierRank'] = None
      data['wins'] = None
      data['losses'] = None
      data['veteran'] = None
      data['inactive'] = None
      data['freshBlood'] = None
      data['hotStreak'] = None
    return data

  def playerStatsLastGames(self, region, summoner, start_index, end_index):
        player = self.lol_watcher.summoner.by_name(region, summoner)
        player_puuid = player['puuid']

        #Fetch recent matchList with Pagination
        matchlist=[]
        matchlist = self.lol_watcher.match.matchlist_by_puuid(region, player_puuid, start_index, end_index)

        # Define the initial values for stats
        total_kills = 0
        total_deaths = 0
        total_assists = 0
        total_gold_earned = 0
        total_matches = 0

        for match in matchlist:
            match_details = self.lol_watcher.match.by_id(region, match)
            for participant in match_details['info']['participants']:
                if participant['puuid'] == player_puuid:
                    total_kills += participant['kills']
                    total_deaths += participant['deaths']
                    total_assists += participant['assists']
                    total_gold_earned += participant['goldEarned']
                    total_matches += 1

        #Calculate the average stats
        average_kills = total_kills / total_matches
        average_deaths = total_deaths / total_matches
        average_assists = total_assists / total_matches
        average_gold_earned = total_gold_earned / total_matches
        data = {
            'average_kills':average_kills,
            'average_deaths': average_deaths,
            'average_assists': average_assists,
            'average_gold_earned': average_gold_earned
            
        }
        return data
    

In [41]:
api_key= "RGAPI-c7cf2424-731f-4a9a-8f1f-f6985e62dac3"
region = 'eun1'
summoner = 'spikal'

karma = Karma(api_key)
summonerStats = karma.player_stats(region, summoner)

In [38]:
statsLastGames = karma.playerStatsLastGames(region, summoner, 0, 30)

In [42]:
data = dict(summonerStats)
data.update(statsLastGames)


In [63]:
def get_random_players(api_key):

    #Retrieve random players for each division

    REGIONS = ["na1", "euw1", "eun1", "kr", "br1", "la1", "la2", "oce", "tr1", "ru"]
    # REGIONS = ["na1", "euw1", "eun1"]
    TIERS = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "DIAMOND"] #, "MASTER", "GRANDMASTER", "CHALLENGER" dont have ranks
    DIVISIONS = ["I", "II", "III", "IV"]
    NUM_PLAYERS_PER_RANK = 100


    lol_watcher = LolWatcher(api_key)
    # Initialize an empty list to store players
    all_players = []
    
    # Loop through regions, tiers, and divisions
    for region in REGIONS:
        for tier in TIERS:
            for division in DIVISIONS:
                # Retrieve player IDs for the current region, tier, and division
                league_entries = lol_watcher.league.entries(region, "RANKED_SOLO_5x5", tier, division)

                # Shuffle the list of players to get random players
                random.shuffle(league_entries)

                # Take the first 100 players (or fewer if there are fewer players available)
                selected_players = league_entries[:NUM_PLAYERS_PER_RANK]

                # Extract player names and regions and add them to the overall list
                players_info = [{"name": entry["summonerName"], "region": region} for entry in selected_players]
                all_players.extend(players_info)

                # Sleep for a short time to avoid rate limiting
                time.sleep(1)  # Adjust the sleep duration as needed

    # Now, all_players_info contains a list of dictionaries with player names and regions
    return all_players #returns a list of dicts



In [64]:
#TODO: Problem with 443 with API
train_data = get_random_players(api_key)

ConnectionError: HTTPSConnectionPool(host='oce.api.riotgames.com', port=443): Max retries exceeded with url: /lol/league/v4/entries/RANKED_SOLO_5x5/IRON/I?page=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001BCED5C6750>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
len(all_players)